In [1]:
# pycaretをimport
from pycaret.classification import *
from pycaret.regression import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')

In [36]:
# ./train/train/*.csvを読み込み、train_dfに格納
train_df = pd.DataFrame()
for file in os.listdir('./train/train/'):
    train_df = pd.concat([train_df, pd.read_csv('./train/train/' + file)], axis=0)
    break

# indexを振り直す
train_df = train_df.reset_index(drop=True)

# test.csvを読み込み、test_dfに格納
test_df = pd.DataFrame()
test_df = pd.read_csv('./test.csv')

train_df.drop(columns=['ID'], inplace=True)
test_id = test_df['ID']
# test_df.drop(columns=['ID'], inplace=True)
columns = train_df.columns.to_list()

features = columns[:-1]
target = columns[-1]
columns

['種類',
 '地域',
 '市区町村コード',
 '都道府県名',
 '市区町村名',
 '地区名',
 '最寄駅：名称',
 '最寄駅：距離（分）',
 '間取り',
 '面積（㎡）',
 '土地の形状',
 '間口',
 '延床面積（㎡）',
 '建築年',
 '建物の構造',
 '用途',
 '今後の利用目的',
 '前面道路：方位',
 '前面道路：種類',
 '前面道路：幅員（ｍ）',
 '都市計画',
 '建ぺい率（％）',
 '容積率（％）',
 '取引時点',
 '改装',
 '取引の事情等',
 '取引価格（総額）_log']

In [21]:
# pycaretでtrain_dfの回帰モデルを作成
reg = setup(data=train_df, target=target, session_id=123, log_experiment=True, experiment_name='house_price')


,Description,Value
0,Session id,123
1,Target,取引価格（総額）_log
2,Target type,Regression
3,Original data shape,"(26123, 27)"
4,Transformed data shape,"(26123, 66)"
5,Transformed train set shape,"(18286, 66)"
6,Transformed test set shape,"(7837, 66)"
7,Ordinal features,1
8,Numeric features,11
9,Categorical features,15


In [22]:
# モデルの比較
best_model = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.1000,0.0240,0.1549,0.8263,0.0202,0.0147,0.7400
rf,Random Forest Regressor,0.1036,0.0261,0.1614,0.8113,0.0211,0.0152,5.2950
gbr,Gradient Boosting Regressor,0.1084,0.0266,0.1631,0.8074,0.0212,0.0159,2.3760
et,Extra Trees Regressor,0.1065,0.0282,0.1678,0.7960,0.0219,0.0157,4.6580
ridge,Ridge Regression,0.1218,0.0324,0.1797,0.7658,0.0233,0.0178,0.4240
br,Bayesian Ridge,0.1218,0.0324,0.1798,0.7658,0.0233,0.0178,0.6430
lr,Linear Regression,0.1218,0.0324,0.1799,0.7654,0.0233,0.0178,2.8320
huber,Huber Regressor,0.1417,0.0410,0.2021,0.7033,0.0259,0.0207,1.1430
knn,K Neighbors Regressor,0.1427,0.0413,0.2030,0.7016,0.0261,0.0208,0.6500
omp,Orthogonal Matching Pursuit,0.1424,0.0416,0.2035,0.6992,0.0261,0.0208,0.6700


Processing:   0%|          | 0/77 [00:00<?, ?it/s]

In [39]:
# test_dfの'2000㎡以上'を2000に変換
test_df['面積（㎡）'] = test_df['面積（㎡）'].apply(lambda x: 2000 if x == '2000㎡以上' else x)
# 作成したモデルでtest_dfの予測を行う
pred_holdouts = predict_model(best_model, data=test_df)
# pred_holdouts['ID'] = test_id
# 予測列の名前を'取引価格（総額）_log'に変更
pred_holdouts.rename(columns={'prediction_label': '取引価格（総額）_log'}, inplace=True)
pred_holdouts.head()

,ID,種類,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,...,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log
0,1000077,中古マンション等,NaN,1101,北海道,札幌市中央区,大通西,西１１丁目,2,３ＬＤＫ,...,NaN,NaN,NaN,商業地域,80.0,600.0,2022年第4四半期,未改装,NaN,7.758915
1,1000129,中古マンション等,NaN,1101,北海道,札幌市中央区,大通西,西１８丁目,0,１Ｒ＋Ｓ,...,NaN,NaN,NaN,商業地域,80.0,400.0,2022年第4四半期,未改装,NaN,6.534864
2,1000130,中古マンション等,NaN,1101,北海道,札幌市中央区,大通西,西１８丁目,2,３ＬＤＫ,...,NaN,NaN,NaN,近隣商業地域,80.0,300.0,2022年第4四半期,未改装,NaN,6.869829
3,1000132,中古マンション等,NaN,1101,北海道,札幌市中央区,大通西,西１８丁目,2,２ＬＤＫ,...,NaN,NaN,NaN,商業地域,80.0,400.0,2022年第3四半期,未改装,NaN,6.941420
4,1000133,中古マンション等,NaN,1101,北海道,札幌市中央区,大通西,西１８丁目,2,１ＤＫ,...,NaN,NaN,NaN,商業地域,80.0,400.0,2022年第3四半期,未改装,NaN,6.546699


In [35]:
# ID,取引価格（総額）_logの形式でcsvを出力
pred_holdouts[['ID', '取引価格（総額）_log']].to_csv('submission.csv', index=False)

In [26]:
# test_dfの中から'2000㎡以上'という文字列データが含まれる行を抽出
filtered_df = test_df[test_df.apply(lambda row: row.astype(str).str.contains('2000㎡以上').any(), axis=1)]
filtered_df


,種類,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,...,今後の利用目的,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等
16267,中古マンション等,NaN,28111,兵庫県,神戸市西区,学園西町,学園都市,9,３ＬＤＫ,2000㎡以上,...,住宅,NaN,NaN,NaN,第１種中高層住居専用地域,60.0,200.0,2022年第4四半期,未改装,NaN


In [41]:
train_df.iloc[:3, :].to_dict()

{'種類': {0: '中古マンション等', 1: '中古マンション等', 2: '中古マンション等'},
 '地域': {0: nan, 1: nan, 2: nan},
 '市区町村コード': {0: 1105, 1: 1107, 2: 1105},
 '都道府県名': {0: '北海道', 1: '北海道', 2: '北海道'},
 '市区町村名': {0: '札幌市豊平区', 1: '札幌市西区', 2: '札幌市豊平区'},
 '地区名': {0: '豊平６条', 1: '琴似１条', 2: '平岸１条'},
 '最寄駅：名称': {0: '学園前(札幌)', 1: '琴似(札幌市営)', 2: '南平岸'},
 '最寄駅：距離（分）': {0: '6', 1: '6', 2: '11'},
 '間取り': {0: '４ＬＤＫ', 1: '３ＬＤＫ', 2: '２ＤＫ'},
 '面積（㎡）': {0: 90, 1: 95, 2: 60},
 '土地の形状': {0: nan, 1: nan, 2: nan},
 '間口': {0: nan, 1: nan, 2: nan},
 '延床面積（㎡）': {0: nan, 1: nan, 2: nan},
 '建築年': {0: '平成9年', 1: '平成16年', 2: '平成20年'},
 '建物の構造': {0: 'ＳＲＣ', 1: 'ＳＲＣ', 2: nan},
 '用途': {0: '住宅', 1: '住宅', 2: '住宅'},
 '今後の利用目的': {0: nan, 1: nan, 2: '住宅'},
 '前面道路：方位': {0: nan, 1: nan, 2: nan},
 '前面道路：種類': {0: nan, 1: nan, 2: nan},
 '前面道路：幅員（ｍ）': {0: nan, 1: nan, 2: nan},
 '都市計画': {0: '第１種住居地域', 1: '商業地域', 2: '第１種低層住居専用地域'},
 '建ぺい率（％）': {0: 60.0, 1: 80.0, 2: 40.0},
 '容積率（％）': {0: 200.0, 1: 400.0, 2: 80.0},
 '取引時点': {0: '2010年第2四半期', 1: '2012年第4四半期', 2: '